In [1]:
import numpy as np 
import os
import sunpy
from sunpy.net import Fido, attrs as SAF
import sunpy.map
from astropy.coordinates import EarthLocation,SkyCoord
import astropy.units as u
import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
def download_sdo(tsdo_init_dwl,tsdo_fina_dwl,Tx, Ty, aiasq):
    """
    This function download and crop SDO cubes

    Last modified: F.J Ordoñez A,  7.01.2023 

    Parameters:
    -----------
    tsdo_init_dwl: datetime.datetime
        Starting time 
    tsdo_fina_dwl: datetime.datetime
        Ennding time
    Tx: float 
        Centre position in X   helio projective coordinates 
    Ty: float 
        Centre position in Y   helio projective coordinates 
    aiasq: float
        widht of desired region
    Returns:
    ----------

    It stores the downloaded data in the corresponding folders 

    Comments:
    ---------
    It can be extended ot oher wavelenghts
    """
    
    ## AIA wavelength
    wl=[94,131,171,193,211,304,335,1600,1700]
    #wl=[304]
    # Make Floders to store downloads.
    os.system('mkdir SDO')
    for i in range(len(wl)):
        os.system('mkdir SDO/{}'.format(wl[i]))
        os.system('mkdir SDO/{}/crop'.format(wl[i]))      
    
    os.system('mkdir SDO/magt')
    os.system('mkdir SDO/magt/crop')
    
    ##Downloading AIA
    
    for item in tqdm.tqdm(wl):
        print('Downloading %i A'%item)
        result = Fido.search(SAF.Time(tsdo_init_dwl, tsdo_fina_dwl), 
                                 SAF.Instrument("aia"), SAF.Wavelength(item*u.angstrom))
        print(result)
        file_download = Fido.fetch(result, path='SDO/{}'.format(item))
        
        if len(result)==0:
            continue
        
        if len(file_download)!=len(result[0]):
            
            print('There was a error in server')
            
        while len(file_download)!=len(result[0]):
            
            print('Continue the download to alternative form')
            
            file_download = Fido.fetch(result, path='SDO/{}'.format(item))
    
        #Saving AIA time sequence
        file_download =sorted(file_download)
        # Load all files into a Map sequence
        tmp = sunpy.map.Map(file_download)
        aia_seq = []
        # cropping 
        print('doing cropping')
        for img in tmp:
            top_right = SkyCoord((Tx+aiasq)*u.arcsec, (Ty+aiasq)*u.arcsec, frame=img.coordinate_frame)
            bottom_left = SkyCoord((Tx-aiasq)*u.arcsec, (Ty-aiasq)*u.arcsec, frame=img.coordinate_frame)
            aia_seq.append(img.submap(bottom_left, top_right=top_right)) 

        new_files = [str('SDO/{}/crop/').format(item)+file_download[a][8:-5]+str('_crop_v1.fits') for a in range(len(file_download))]
        for img, file_name in zip(aia_seq, new_files):
            img.save(file_name)
    del wl,result, file_download,tmp,aia_seq,new_files,img 
   #--------------------------------------------------------------------------------------------------
    
    ###Downloading Magnetograms
    wl = ['LOS_magnetic_field']
    print('Donwloading SDO/HMI data')
    for item in tqdm.tqdm(wl):
        print('Downloading %s'%item)
        result = Fido.search(SAF.Time(tsdo_init_dwl, tsdo_fina_dwl), 
                             SAF.Instrument("hmi"), SAF.Physobs(item), 
                             SAF.Sample(45*u.second))    
        print(result)
        file_download = Fido.fetch(result, path='SDO/magt',site='ROB')
        
        if len(file_download)!=len(result[0]):    
            print('There was a error in server')
            
        while len(file_download)!=len(result[0]):
            print('Continue the download to alternative form')
            file_download = Fido.fetch(result, path='SDO/magt',site='ROB')
        
        
        #Saving HMI time sequence
        file_download = sorted(file_download)
        ###Load all files in map sequence
        tmp = sunpy.map.Map(file_download)
        #Cropping defined area
        
        # In this case, we do not run aiaprep because we're only using one channel
        aia_seq = []  #It could be called hmi_seq but it is just a variable
        print('cropping img')
        for img in tmp:
            top_right = SkyCoord((Tx+aiasq)*u.arcsec, (Ty+aiasq)*u.arcsec, frame=img.coordinate_frame)
            bottom_left = SkyCoord((Tx-aiasq)*u.arcsec, (Ty-aiasq)*u.arcsec, frame=img.coordinate_frame)
            aia_seq.append(img.submap(bottom_left, top_right=top_right))
        new_files =  [str('SDO/magt/crop')+file_download[a][8:-5]+str('_crop_v1.fits') for a in range(len(file_download))]
        for img, file_name in zip(aia_seq, new_files):
            img.save(file_name)
    
    return print('The download process for SDO and HMIhasve finished.')

In [3]:
kk=download_sdo('2018-04-12T15:35:28','2018-04-12T16:30:21',-128,-400,140) # DELTA IMPORTANTE  

mkdir: cannot create directory ‘SDO’: File exists
mkdir: cannot create directory ‘SDO/magt’: File exists
mkdir: cannot create directory ‘SDO/magt/crop’: File exists
  0%|                                                     | 0/1 [00:00<?, ?it/s]

Results from 1 Provider:

275 Results from the VSOClient:
Source: http://vso.stanford.edu/cgi-bin/search
Total estimated size: 18.642 Gbyte

       Start Time       ...
                        ...
----------------------- ...
2018-04-12 15:35:29.000 ...
2018-04-12 15:35:41.000 ...
2018-04-12 15:35:53.000 ...
2018-04-12 15:36:05.000 ...
2018-04-12 15:36:17.000 ...
2018-04-12 15:36:29.000 ...
                    ... ...
2018-04-12 16:29:05.000 ...
2018-04-12 16:29:17.000 ...
2018-04-12 16:29:29.000 ...
2018-04-12 16:29:41.000 ...
2018-04-12 16:29:53.000 ...
2018-04-12 16:30:05.000 ...
2018-04-12 16:30:17.000 ...
Length = 275 rows




Files Downloaded:   0%|          | 0/275 [00:00<?, ?file/s]

aia_lev1_304a_2018_04_12t15_35_53_15z_image_lev1.fits:   0%|          | 0.00/7.27M [00:00<?, ?B/s]

aia_lev1_304a_2018_04_12t15_36_05_13z_image_lev1.fits:   0%|          | 0.00/7.27M [00:00<?, ?B/s]

...

Results from 1 Provider:

73 Results from the VSOClient:
Source: http://vso.stanford.edu/cgi-bin/search

       Start Time               End Time        ...   Size          Info       
                                                ...  Mibyte                    
----------------------- ----------------------- ... -------- ------------------
2018-04-12 15:35:46.000 2018-04-12 15:35:47.000 ... -0.00098 45sec. Magnetogram
2018-04-12 15:36:31.000 2018-04-12 15:36:32.000 ... -0.00098 45sec. Magnetogram
2018-04-12 15:37:16.000 2018-04-12 15:37:17.000 ... -0.00098 45sec. Magnetogram
2018-04-12 15:38:01.000 2018-04-12 15:38:02.000 ... -0.00098 45sec. Magnetogram
2018-04-12 15:38:46.000 2018-04-12 15:38:47.000 ... -0.00098 45sec. Magnetogram
2018-04-12 15:39:31.000 2018-04-12 15:39:32.000 ... -0.00098 45sec. Magnetogram
                    ...                     ... ...      ...                ...
2018-04-12 16:25:16.000 2018-04-12 16:25:17.000 ... -0.00098 45sec. Magnetogram
2018-04-12 16:2

Files Downloaded:   0%|          | 0/73 [00:00<?, ?file/s]

cropping img


  0%|                                                     | 0/1 [00:57<?, ?it/s]


OSError: File SDO/magt/crop/hmi_20180412_153546_magnetogram_crop_v1.fits already exists. If you mean to replace it then use the argument "overwrite=True".

###  In this case, we wnat to study D06 region of the SALSA data base 
#(ALMA Data [url: http://sdc.uio.no/salsa/]) for that we took T_x, T_y = -128,-400 as center of our  center helio projective coordinates, and took 80 as radius because we want to has a window (FoV) bigger than the alma FoV.

In [6]:

kk=download_sdo('2018-04-12T15:45:28','2018-04-12T16:30:21',-128,-400,80) # DELTA IMPORTANTE  

  0%|                                                     | 0/1 [00:00<?, ?it/s]

Results from 1 Provider:

111 Results from the VSOClient:
Source: http://vso.stanford.edu/cgi-bin/search
Total estimated size: 7.525 Gbyte

       Start Time       ...
                        ...
----------------------- ...
2018-04-12 15:45:40.000 ...
2018-04-12 15:46:04.000 ...
2018-04-12 15:46:28.000 ...
2018-04-12 15:46:52.000 ...
2018-04-12 15:47:16.000 ...
2018-04-12 15:47:40.000 ...
                    ... ...
2018-04-12 16:27:40.000 ...
2018-04-12 16:28:04.000 ...
2018-04-12 16:28:28.000 ...
2018-04-12 16:28:52.000 ...
2018-04-12 16:29:16.000 ...
2018-04-12 16:29:40.000 ...
2018-04-12 16:30:04.000 ...
Length = 111 rows




Files Downloaded:   0%|          | 0/111 [00:00<?, ?file/s]

aia_lev1_1700a_2018_04_12t15_46_52_72z_image_lev1.fits:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

aia_lev1_1700a_2018_04_12t15_46_04_72z_image_lev1.fits:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

...

hmi_20180412_162901_magnetogram.fits: 0.00B [00:00, ?B/s]

hmi_20180412_162946_magnetogram.fits: 0.00B [00:00, ?B/s]

cropping img


100%|████████████████████████████████████████████| 1/1 [02:21<00:00, 141.69s/it]

The download process for SDO and HMIhasve finished.


In [ ]:
sumpy.map.Map('/home/javier/Documents/MSc-program/SDO_DATA_ALIGNIING/SDO/94/crop/ia_lev1_94a_2018_04_12t15_35_35_13z_image_lev1_crop_v1.fits')